In [1]:
using JuMP, Gurobi, Concorde
import GLPK
import Random
import Plots

Gerenrate distance matrix

In [87]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    d = [round(Int, sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2)) for i in 1:n, j in 1:n]
    return X, Y, d
end

n = 4
X, Y, d = generate_distance_matrix(n)
d

4×4 Matrix{Int64}:
  0  30  47  56
 30   0  74  86
 47  74   0  31
 56  86  31   0

Bitmask를 이용하여 어떤 도시들을 지나왔는지 기록

In [88]:
i = 5 # Replace this with your desired value of i
result = (1 << (i-1)) -1

println("Result in binary: ", string(result, base=2))

Result in binary: 1111


Forward DP with Trajectory

In [89]:
function tsp_dynamic_programming_forward()
    memo = fill((Inf, 1), n, 1 << n) # (cost, last_pos), (current_pos, mask)
    memo[1, 1] = (0, 0)
    
    for mask in 1:(1 << n) - 1
        for s in 1:n
            if (mask & (1 << (s - 1))) != 0
                for j in 1:n
                    if (mask & (1 << (j - 1))) == 0
                        if memo[j, mask | (1 << (j - 1))][1] > memo[s, mask][1] + d[s, j]
                            memo[j, mask | (1 << (j - 1))] = (memo[s, mask][1] + d[s, j], s)
                        end
                    end
                end
            end
        end
    end

    # find answer
    ans = 10^9
    last_pos = 0
    for i in 2:n
        if ans > memo[i, (1 << n) - 1][1] + d[i, 1]
            ans = memo[i, (1 << n) - 1][1] + d[i, 1]
            last_pos = i
        end
    end
    
    # find trajectory
    trajectory = [last_pos]
    last_mask = (1 << n) - 1
    for i in 1:n-1
        push!(trajectory, memo[trajectory[end], last_mask][2])
        last_mask = last_mask & ~ (1 << (trajectory[end-1]-1))
    end
    trajectory = reverse(trajectory)

    return trajectory, ans
end


result_trajectory, result_ans = tsp_dynamic_programming_forward()

# Print the result
println("tour: ", result_trajectory)
println("cost: ", result_ans)


tour: [1, 4, 3, 2]
cost: 191.0


Backward DP with Trajectory

In [99]:
function tsp_dynamic_programming_backward(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    memo = Dict{Tuple{Int, Int}, Tuple{Int, Int}}() # key: (mask, current_pos), value: (cost, next_pos)
    trajectory = []
    result, traj = tsp(memo, distance_matrix, 1, 1, n, trajectory)

    #=
    # print memo 
    for (key, value) in pairs(memo)
        binary_mask = bitstring(key[1])[end-3:end]
        println("Key: ($binary_mask, $(key[2])), Value: $value")
    end
    =#

    return result, traj
end

function tsp(memo, distance_matrix, mask, pos, n, trajectory)
    if haskey(memo, (mask, pos)) # already computed
        return memo[(mask, pos)]
    end

    if mask == 2^n - 1 # all cities visited
        return (distance_matrix[pos, 1], 1)
    end

    min_cost = Inf
    next_pos = 0
    for next_city in 2:n
        if (mask & (1 << (next_city - 1))) == 0 # next_city is not visited.
            new_mask = mask | (1 << (next_city - 1))
            new_cost = distance_matrix[pos, next_city] + tsp(memo, distance_matrix, new_mask, next_city, n, trajectory)[1][1]
            if min_cost > new_cost
                next_pos = next_city
            end
            min_cost = min(min_cost, new_cost)
        end
    end
    push!(trajectory, next_pos)
    memo[(mask, pos)] = (min_cost, next_pos)
    mask = bitstring(mask)[end-3:end]
    # print memo
    println("mask: $mask, pos: $pos")
    for (key, value) in pairs(memo)
        binary_mask = bitstring(key[1])[end-3:end]
        println("Key: ($binary_mask, $(key[2])), Value: $value")
    end

    return (min_cost, next_pos), trajectory
end

distances = d

result, traj = tsp_dynamic_programming_backward(distances)
println("cost: $(result[1])")
println("tour: ", [1; traj])

d

mask: 0111, pos: 3
Key: (0111, 3), Value: (87, 4)
mask: 1011, pos: 4
Key: (1011, 4), Value: (78, 3)
Key: (0111, 3), Value: (87, 4)
mask: 0011, pos: 2
Key: (1011, 4), Value: (78, 3)
Key: (0011, 2), Value: (161, 3)
Key: (0111, 3), Value: (87, 4)
mask: 0111, pos: 2
Key: (1011, 4), Value: (78, 3)
Key: (0011, 2), Value: (161, 3)
Key: (0111, 2), Value: (142, 4)
Key: (0111, 3), Value: (87, 4)
mask: 1101, pos: 4
Key: (1011, 4), Value: (78, 3)
Key: (0011, 2), Value: (161, 3)
Key: (0111, 2), Value: (142, 4)
Key: (0111, 3), Value: (87, 4)
Key: (1101, 4), Value: (116, 2)
mask: 0101, pos: 3
Key: (1011, 4), Value: (78, 3)
Key: (0011, 2), Value: (161, 3)
Key: (0111, 2), Value: (142, 4)
Key: (0101, 3), Value: (147, 4)
Key: (0111, 3), Value: (87, 4)
Key: (1101, 4), Value: (116, 2)
mask: 1011, pos: 2
Key: (1011, 4), Value: (78, 3)
Key: (0011, 2), Value: (161, 3)
Key: (0111, 2), Value: (142, 4)
Key: (1011, 2), Value: (121, 3)
Key: (0101, 3), Value: (147, 4)
Key: (0111, 3), Value: (87, 4)
Key: (1101, 4), 

4×4 Matrix{Int64}:
  0  30  47  56
 30   0  74  86
 47  74   0  31
 56  86  31   0

Concorde

In [81]:
opt_tour, opt_len = solve_tsp(X, Y; dist="EUC_2D")

([1, 4, 3, 2], 191)

Recursion

In [8]:
#=
s: starting point 
mask: 지나갔는지 안지나갔는지 확인 
=#

# distance matrix
d =[
    0  10 15 20;
    10 0  35 25;
    15 35 0  30;
    20 25 30 0
]

# define n
rows , cols = size(d)
n = rows

# memo
memo = fill(Inf, (n, 1 << n))

function are_two_set_bits(mask)
    count_set_bits = 0
    while mask > 0
        count_set_bits += mask & 1
        mask >>= 1
    end
    return count_set_bits == 2
end

function indices_of_set_bits(mask)
    indices = Int[]
    position = 1
    
    while mask > 0
        if mask & 1 == 1
            push!(indices, position)
        end
        mask >>= 1
        position += 1
    end
    return indices
end

function tsp_dynamic_programming(mask, s)
    # s는 지나감
    mask = mask | (1 << (s-1))
    
    # base case, 먼가 여기서 잘못된거 같은데... base case어떻게 고쳐야 할까요? -> 맞게 고친것 같은데 
    indices = indices_of_set_bits(mask)
    if are_two_set_bits(mask)
        memo[s, mask] = 0
    end

    #memoization
    if memo[s, mask] != Inf
        return memo[s, mask]
    end
    res = Inf
    for i in 1:n
        if mask & (1 << (i-1)) == 0 && i != s
            updated_mask = mask | (1 << (i-1))
            val = tsp_dynamic_programming(updated_mask, i) + d[i, s]
            res = val # res 값이 업데이트가 되지 않는다. 
        end
    end
    memo[s, mask] = res
    
    return res
end

ans = Inf
for j in 1:n
    ans = min(ans, tsp_dynamic_programming((1 << (n-1)) - 1, j) + d[1, j])
end
println("The cost of most efficient tour = $ans")

The cost of most efficient tour = Inf
